# **Yoga** **AI**


In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import json
import string
import os

## **DATA COLLECTION**

In [ ]:
url = "https://pocketyoga.com/poses.json"

response = requests.get(url)

#pulls data from json file and saves it as a csv file titles "yoga_poses.csv"

if response.status_code == 200:
  data = response.json()

  processed_data = []
  for pose in data:

    #Gets data from the JSON file and stores it under pose_data
    pose_data = {
        'Display Name' : pose.get('display_name', 'N/A'),
        'Name' : pose.get('name', 'N/A'),
        'Category' : pose.get('category', 'N/A'),
        'Sub Category' : pose.get('subcategory', 'N/A'),
        'Difficulty' : pose.get('difficulty', 'N/A'),
        'Benefits' : pose.get('benefits', 'N/A'),
        'Description' : pose.get('description', 'N/A'),
        'Preferred Side' : pose.get('preferred_side', 'N/A'),
        'Next Position' : pose.get('next_poses', 'N/A')
    }
    #appends the data, and stores it under processed_data
    processed_data.append(pose_data)


  #Converts Processed data into a DataFrame
  df = pd.DataFrame(processed_data)

  #Saves df as a csv file
  df.to_csv('yoga_poses.csv', index=False)
  print("Data has been saved to 'yoga_poses.csv'.")
  print('here')
else:
  print(f"Failed to retrieve the JSON data. Status code: {response.status_code}")

Data has been saved to 'yoga_poses.csv'.
here


## **DATA CLEANING**

In [ ]:
#remove null rows
df_cleaned = df.dropna()

In [ ]:
df_cleaned.head()

,Display Name,Name,Category,Sub Category,Difficulty,Benefits,Description,Preferred Side,Next Position
0,Archer's,Archer,seated,neutral,intermediate,"Stretches the arms, hips and legs. Strengthen...",Begin in seated position with your legs straig...,left,[Seated Forward Bend Preparation]
2,Awkward Airplane,Awkward Airplane,arm_leg_support,neutral,beginner,Strengthens the core and hip mobility. Improv...,Begin in //Cat Balance//. Send your floating ...,right,[Cat Balance]
3,Banana,Banana,supine,lateral_bend,beginner,Strengthens the spine and rib cage in a latera...,Start from //Corpse// pose. Bring your hands ...,left,[Corpse Extended]
4,Revolved Bird of Paradise,Bird Of Paradise Revolved,standing,balancing,expert,Increases the flexibility of the spine and bac...,"From //Chair Twist// pose, the lower arm reach...",left,[Bird Of Paradise Revolved Preparation]
21,Kneeling Bow,Bow Half,arm_leg_support,backbend,intermediate,Stretches the back muscles and the front body....,"From //Box Neutral//, extend one arm forward a...",right,"[Box Neutral, Cat Balance]"


In [ ]:
keywords = {
    'arms': 'arms',
    'hips': 'hips',
    'legs': 'legs',
    'hip': 'hip',
    'shoulders': 'shoulders',
    'pelvis': 'pelvis',
    'ankles': 'ankles',
    'core': 'core',
    'spine': 'spine',
    'rib cage': 'rib cage',
    'back': 'back',
    'hamstrings': 'hamstrings',
    'knees': 'knees',
    'thighs': 'thighs',
    'chest': 'chest',
    'neck': 'neck',
    'feet': 'feet',
    'calves': 'calves',
    'wrists': 'wrists',
    'hands': 'hands',
    'glutes': 'glutes'
}

def extract_keywords(benefits, keywords):
    if benefits is None:
        return None
    found_keywords = [muscle for muscle in keywords if muscle in benefits.lower()]
    return ', '.join(found_keywords) if found_keywords else None

df_cleaned['Body Part Strengthened'] = df_cleaned['Benefits'].apply(lambda x: extract_keywords(x, keywords))


<ipython-input-6-7260783b7bd2>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Body Part Strengthened'] = df_cleaned['Benefits'].apply(lambda x: extract_keywords(x, keywords))


In [ ]:
null_counts = df_cleaned.isnull().sum()

print(null_counts)

Display Name              0
Name                      0
Category                  0
Sub Category              0
Difficulty                0
Benefits                  0
Description               0
Preferred Side            0
Next Position             0
Body Part Strengthened    5
dtype: int64


In [ ]:
df = df_cleaned.dropna()
df = df.applymap(lambda x: x.lower().translate(str.maketrans('', '', string.punctuation))if isinstance(x, str) else x)

df.head()

<ipython-input-8-65326c16dc05>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().translate(str.maketrans('', '', string.punctuation))if isinstance(x, str) else x)


,Display Name,Name,Category,Sub Category,Difficulty,Benefits,Description,Preferred Side,Next Position,Body Part Strengthened
0,archers,archer,seated,neutral,intermediate,stretches the arms hips and legs strengthens ...,begin in seated position with your legs straig...,left,[Seated Forward Bend Preparation],arms hips legs hip shoulders
2,awkward airplane,awkward airplane,armlegsupport,neutral,beginner,strengthens the core and hip mobility improve...,begin in cat balance send your floating leg o...,right,[Cat Balance],hip core
3,banana,banana,supine,lateralbend,beginner,strengthens the spine and rib cage in a latera...,start from corpse pose bring your hands overh...,left,[Corpse Extended],spine rib cage
4,revolved bird of paradise,bird of paradise revolved,standing,balancing,expert,increases the flexibility of the spine and bac...,from chair twist pose the lower arm reaches ba...,left,[Bird Of Paradise Revolved Preparation],legs hip shoulders spine back hamstrings
21,kneeling bow,bow half,armlegsupport,backbend,intermediate,stretches the back muscles and the front body ...,from box neutral extend one arm forward and th...,right,"[Box Neutral, Cat Balance]",back


In [ ]:
#Outputs the unique values in Category Column
unique_values = df['Category'].unique()
print(unique_values)

['seated' 'armlegsupport' 'supine' 'standing' 'prone'
 'armbalanceandinversion']


In [ ]:
df_seated = df[df['Category'] == 'seated']
df_armlegsupport = df[df['Category'] == 'armlegsupport']
df_supine = df[df['Category'] == 'supine']
df_standing = df[df['Category'] == 'standing']
df_prone = df[df['Category'] == 'prone']
df_armbalanceandinversion = df[df['Category'] == 'armbalanceandinversion']

In [ ]:
#converts dfs into a json file
df_seated.to_json('Seated.json', orient='records', lines=True)
df_armlegsupport.to_json('Armlegsupport.json', orient='records', lines=True)
df_supine.to_json('Supine.json', orient='records', lines=True)
df_standing.to_json('Standing.json', orient='records', lines=True)
df_prone.to_json('Prone.json', orient='records', lines=True)
df_armbalanceandinversion.to_json('Armbalanceandinversion.json', orient='records', lines=True)

In [ ]:
#function to find errors in the Json file
def inspect_json_file(json_file):
    try:
        with open(json_file, 'r') as file:
            content = file.read()
            print(content)
            data = json.loads(content) #parses json file
            print(data)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")

inspect_json_file('Seated.json')
inspect_json_file('Armlegsupport.json')
inspect_json_file('Supine.json')
inspect_json_file('Standing.json')
inspect_json_file('Prone.json')
inspect_json_file('Armbalanceandinversion.json')

{"Display Name":"archers","Name":"archer","Category":"seated","Sub Category":"neutral","Difficulty":"intermediate","Benefits":"stretches the arms hips and legs  strengthens the arms shoulders and hip mobility","Description":"begin in seated position with your legs straight in front and hinge forward  wrap your index and middle fingers around your big toes  bend one knee and place the heel on the floor a few inches away from your opposite inner knee  press the back of your straight leg into the floor then hinge slightly forward  keep a strong hold of both big toes and pick up the leg of your bent knee  you may need to lean your weight slightly over toward the straight leg  draw your elbow and shoulder back to bring your foot close to your ear and then root both sitting bones as much as possible","Preferred Side":"left","Next Position":["Seated Forward Bend Preparation"],"Body Part Strengthened":"arms hips legs hip shoulders"}
{"Display Name":"rock the baby","Name":"cradle","Category":"s

In [ ]:
#This funtion converts the josn file into a readable format for md conversion.
def load_multiple_json_objects(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        json_objects = content.splitlines()
        data = []
        for obj in json_objects:
            if obj.strip():
                try:
                    data.append(json.loads(obj))
                except json.JSONDecodeError as e:
                    print(f"Skipping invalid JSON object: {obj}\nError: {e}")
        return data

In [ ]:
#This function converts json file into a Markdown file.
def json_to_markdown_list(json_file, output_file):
    data = load_multiple_json_objects(json_file)
    base_name = os.path.basename(json_file)
    title = os.path.splitext(base_name)[0]

    with open(output_file, 'w') as file:
        file.write(f"{title} Poses\n\n")
        for item in data:
            file.write(f"Name: {item['Name']}\n")
            file.write(f"Category: {item['Category']}\n")
            file.write(f"Body Part Strengthened: {item['Body Part Strengthened']}\n\n")
            file.write(f"Difficulty: {item['Difficulty']}\n\n")
            file.write(f"Benefits: {item['Benefits']}\n\n")
            file.write(f"Description: {item['Description']}\n\n")
            file.write(f"Next Position: {item['Next Position']}\n\n")

#applys function to 'yoga_poses_cleaned.json' file and outputs as 'yoga_poses_list.md'
json_to_markdown_list('Seated.json', 'df_seated.md')
json_to_markdown_list('Armlegsupport.json', 'df_armlegsupport.md')
json_to_markdown_list('Supine.json', 'df_supine.md')
json_to_markdown_list('Standing.json', 'df_standing.md')
json_to_markdown_list('Prone.json', 'df_prone.md')
json_to_markdown_list('Armbalanceandinversion.json', 'df_armbalanceandinversion.md')